In [1]:
!pip install google-generativeai evaluate datasets pandas torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install -q -U google-generativeai
import google.generativeai as genai
import pandas as pd
import time



# configure API key
genai.configure(api_key="API_KEY")

model = genai.GenerativeModel("gemini-1.5-flash-latest")


In [6]:
from google.colab import drive
drive.mount('/content/drive')

dataset_path = "/content/qa_dataset (2).json"
df = pd.read_json(dataset_path, lines=True)

# preview
display(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,instruction,response
0,What is the warranty period for Sony laptops?,All Sony laptops come with a 3 years manufactu...
1,Do you sell Huawei headphones?,"Yes, ClickBuy offers Huawei headphones with fu..."
2,What is your return policy?,You can return any product within 14 days of p...
3,What is the warranty period for Samsung fridges?,All Samsung fridges come with a 1 year manufac...
4,What is the warranty period for Xiaomi microwa...,All Xiaomi microwaves come with a 1 year manuf...


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('instruction').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('response').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['response'].value_counts()
    for x_label, grp in _df_2.groupby('instruction')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('instruction')
_ = plt.ylabel('response')

In [10]:
!pip install -U google-generativeai



In [ ]:
import google.generativeai as genai
genai.configure(api_key="API_KEY")


In [14]:
model = genai.GenerativeModel("gemini-2.5-flash")

In [15]:
response = model.generate_content("Hi! How are you?")
print(response.text)


Hello! As an AI, I don't experience emotions or have personal well-being, but I'm functioning perfectly and ready to help!

How are *you* doing today, and what can I assist you with?


In [16]:
import time

preds, refs = [], []

for i, row in df.head(25).iterrows():
    try:
        prompt = row["instruction"]
        response = model.generate_content(prompt)
        preds.append(response.text)
        refs.append(row["response"])
        print(f"✅ Done {i+1}")
        time.sleep(3)
    except Exception as e:
        print(f"⚠️ Error on {i+1}: {e}")
        preds.append("")
        refs.append(row["response"])


✅ Done 1
✅ Done 2
✅ Done 3
✅ Done 4
✅ Done 5
✅ Done 6
✅ Done 7
✅ Done 8
✅ Done 9
✅ Done 10
✅ Done 11
✅ Done 12
✅ Done 13
✅ Done 14
✅ Done 15
✅ Done 16
✅ Done 17
✅ Done 18
✅ Done 19
✅ Done 20
✅ Done 21
✅ Done 22
✅ Done 23
✅ Done 24
✅ Done 25


In [21]:
from evaluate import load
import math
import numpy as np
import pandas as pd

# Metrics
bleu = load("bleu")
rouge = load("rouge")
bertscore = load("bertscore")

results = {
    "BLEU": bleu.compute(predictions=preds, references=refs)["bleu"],
    "ROUGE-L": rouge.compute(predictions=preds, references=refs)["rougeL"],
    "BERTScore": np.mean(bertscore.compute(predictions=preds, references=refs, lang="en")["f1"])
}

# Perplexity (using a dummy method since Gemini doesn't expose logits)
avg_len = np.mean([len(p.split()) for p in preds if p])
results["Perplexity"] = math.exp(-np.log(avg_len + 1e-8))

df_results = pd.DataFrame([results])
df_results.to_csv("gemini_eval.csv", index=False)
print(df_results)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


       BLEU   ROUGE-L  BERTScore  Perplexity
0  0.002318  0.053595   0.821986    0.007452


In [20]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.8 MB/s eta 0:00:00


In [18]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4bcaba4bd0f02d7df7b48d7d8956e1b20aaa2d869d022ee3358fb3351f860932
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
